# Export for Dashboard

In [102]:
# For multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [79]:
#DATASET_FOLDER = '/media/data-nvme/dev/datasets/WorldBank/'
DATASET_FOLDER = '../../datasets/precipitation/'

SPARK_MASTER = 'spark://192.168.0.9:7077'
APP_NAME = 'Merge PRCP GKP'
input_folder = DATASET_FOLDER
output = DATASET_FOLDER + '../wb_gkp_precipitation'

In [80]:
import os
import pandas as pd

In [145]:
import pandas as pd
from pyspark import SparkContext
#from pyspark import SparkConf
from pyspark.sql import SparkSession
#from pyspark.sql.window import Window
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

import shutil

## Connect to Spark

In [116]:
print('Create Spark session')
spark = SparkSession.builder.master(SPARK_MASTER).appName(APP_NAME).getOrCreate()
sc = spark.sparkContext

Create Spark session


In [1]:
#!ls /media/data-nvme/dev/datasets/WorldBank/

In [126]:
emdat = spark.read.format('csv').option('header',True).option('multiLine', True).load('/media/data-nvme/dev/datasets/WorldBank/' + 'emdat_public_2020_09_12_query_uid-tAnKEX-floods_only.csv')

In [143]:
emdat.columns

['Dis No',
 'Year',
 'Seq',
 'Disaster Subtype',
 'Disaster Subsubtype',
 'Event Name',
 'Entry Criteria',
 'Country',
 'ISO',
 'Region',
 'Continent',
 'Location',
 'Origin',
 'Associated Dis',
 'Associated Dis2',
 'OFDA Response',
 'Appeal',
 'Declaration',
 'Aid Contribution',
 'Dis Mag Value',
 'Dis Mag Scale',
 'Latitude',
 'Longitude',
 'Local Time',
 'River Basin',
 'Total Deaths',
 'No Injured',
 'No Affected',
 'No Homeless',
 'Total Affected',
 "Reconstruction Costs ('000 US$)",
 "Insured Damages ('000 US$)",
 "Total Damages ('000 US$)",
 'CPI',
 'date',
 'decade']

In [191]:
#emdat = emdat.withColumn('decade',  F.concat(F.col('Year').substr(0, 3) , F.lit('0-') , F.col('Year').substr(0, 3), F.lit('9')))
emdat = emdat.withColumn('decade',  F.concat(F.col('Year').substr(0, 3) , F.lit('0')))
emdat.take(3)

[Row(Dis No='1906-0023-BEL', Year='1906', Seq='23', Disaster Subtype=None, Disaster Subsubtype=None, Event Name=None, Entry Criteria='Govern', Country='Belgium', ISO='BEL', Region='Western Europe', Continent='Europe', Location='Louvain region', Origin=None, Associated Dis=None, Associated Dis2=None, OFDA Response=None, Appeal=None, Declaration=None, Aid Contribution=None, Dis Mag Value=None, Dis Mag Scale='Km2', Latitude=None, Longitude=None, Local Time=None, River Basin=None, Total Deaths=6.0, No Injured=None, No Affected=None, No Homeless=None, Total Affected=None, Reconstruction Costs ('000 US$)=None, Insured Damages ('000 US$)=None, Total Damages ('000 US$)=None, CPI='3.5223001018494', date='1906-5-14', decade='1900'),
 Row(Dis No='1906-0024-BEL', Year='1906', Seq='24', Disaster Subtype=None, Disaster Subsubtype=None, Event Name=None, Entry Criteria='Govern', Country='Belgium', ISO='BEL', Region='Western Europe', Continent='Europe', Location=None, Origin=None, Associated Dis=None, 

In [192]:
###### Cast Type
print('Cast string to float')
emdat = emdat.withColumn("Total Deaths", emdat["Total Deaths"].cast(FloatType()))\
    .withColumn("Total Damages ('000 US$)", emdat["Total Damages ('000 US$)"].cast(FloatType()))
emdat.createOrReplaceTempView("emdat")


Cast string to float


In [193]:
emdat.count()

5226

In [194]:
#dfp = emdat.toPandas()

In [195]:
#dfp[['Dis No', "Total Deaths"]]

In [196]:
#dfp[dfp['Dis No'] == '1906-0023-BEL'] #") #1906-0023-BEL 	1950-0007-CHN

In [197]:
# spark.sql("""
# SELECT *
# FROM emdat
# WHERE `Dis No` == '1950-0007-CHN'
# """).show(3)

In [198]:
# spark.sql("""
# SELECT *
# FROM emdat
# WHERE `Dis No` LIKE '%BEL%'
# """).show(3)

In [213]:
#from pyspark.sql.window import Window
#windowSpec = Window.partitionBy(['Region']).orderBy(F.col("Year"))
#df_rolling = emdat.withColumn('last_'+str(d)+'_days_sum', F.sum("Total Damages ('000 US$)").over(windowSpec.rangeBetween(-days(d), 0)))
emdat_agregate = spark.sql("""
SELECT decade as Decade, region as UN_Geosheme_Subregion, 'Flood' as Disaster_Type, 'past' as RCP, sum(`Total Damages ('000 US$)`) as Financial_Impact, sum(`Total Deaths`) as Human_Impact, count(`Dis No`) as DO, '0' as Rain
FROM emdat
GROUP BY decade, region
ORDER BY decade, region
""")
dfp = emdat_agregate.toPandas()
dfp.dropna()

,Decade,UN_Geosheme_Subregion,Disaster_Type,RCP,Financial_Impact,Human_Impact,DO,Rain
5,1920,Northern America,Flood,past,230.0,246.0,1,0
10,1930,Eastern Asia,Flood,past,1400000.0,4200933.0,3,0
11,1930,Northern America,Flood,past,438000.0,337.0,2,0
14,1940,Central America,Flood,past,15000.0,40000.0,1,0
24,1950,Northern America,Flood,past,1129000.0,149.0,4,0
...,...,...,...,...,...,...,...,...
154,2020,Eastern Asia,Flood,past,6100000.0,256.0,5,0
158,2020,Northern Africa,Flood,past,76000.0,130.0,2,0
159,2020,South America,Flood,past,157000.0,195.0,8,0
160,2020,South-Eastern Asia,Flood,past,28900.0,87.0,16,0


In [214]:
dfp = dfp.fillna({'Financial_Impact' : 0, 'Human_Impact' : 0})

In [215]:
dfp.query("Human_Impact > 0")

,Decade,UN_Geosheme_Subregion,Disaster_Type,RCP,Financial_Impact,Human_Impact,DO,Rain
0,1900,Caribbean,Flood,past,0.0,300.0,1,0
1,1900,Northern America,Flood,past,0.0,72.0,1,0
2,1900,Western Europe,Flood,past,0.0,6.0,2,0
3,1910,Eastern Asia,Flood,past,0.0,1379.0,1,0
4,1920,Northern Africa,Flood,past,0.0,3000.0,1,0
...,...,...,...,...,...,...,...,...
159,2020,South America,Flood,past,157000.0,195.0,8,0
160,2020,South-Eastern Asia,Flood,past,28900.0,87.0,16,0
161,2020,Southern Asia,Flood,past,1300000.0,1870.0,20,0
163,2020,Western Africa,Flood,past,0.0,63.0,3,0


In [216]:
emdat_agregate.createOrReplaceTempView("emdat_agregate")

spark.sql("""
SELECT *
FROM emdat_agregate
WHERE Financial_Impact > 0
""").show(3)

+------+---------------------+-------------+----+----------------+------------+---+----+
|Decade|UN_Geosheme_Subregion|Disaster_Type| RCP|Financial_Impact|Human_Impact| DO|Rain|
+------+---------------------+-------------+----+----------------+------------+---+----+
|  1920|     Northern America|        Flood|past|           230.0|       246.0|  1|   0|
|  1920|       Western Europe|        Flood|past|         70000.0|        null|  3|   0|
|  1930|         Eastern Asia|        Flood|past|       1400000.0|   4200933.0|  3|   0|
+------+---------------------+-------------+----+----------------+------------+---+----+
only showing top 3 rows



In [217]:
#emdat_agregate.write.csv(f'{DATASET_FOLDER}../flood_history_agregates.csv', header=True)
dfp.to_csv(f'{DATASET_FOLDER}../flood_history_agregates.csv.gz', index=False ,compression='gzip')

In [ ]:
sc.stop()